In [1]:
# !pip install selenium

In [2]:
# !pip install bs4

In [1]:
# pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


install Microsoft Edge WebDriver

In [1]:
import xlsxwriter
from selenium import webdriver

In [2]:
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import json
import re
import pandas as pd
import requests
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options

In [3]:
edge_options = Options()
edge_options.use_chromium = True  # This line is important
# edge_options.add_argument("--headless")
edge_options.add_argument("--disable-gpu")
edge_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.54"
)
edge_options.add_argument("window-size=1920x1080")
edge_options.add_argument("disable-extensions")
edge_options.add_argument("--disable-blink-features=AutomationControlled")
# edge_options.add_experimental_option("excludeSwitches", ["enable-automation"])

### Lazada

In [20]:
# Function to scrape review JSON from the webpage
def scrape_review(driver):
    # After CAPTCHA is solved, proceed to the next step
    response = driver.execute_script("return document.body.innerHTML;")
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(response, 'html.parser')
    
    # Extract the hidden JSON data from the <pre> tag
    hidden_div = soup.find("pre")
    return hidden_div

In [21]:
itemid = 5099789889
# Initialize an empty DataFrame to store all reviews
df = pd.DataFrame()

# Initialize the Selenium Edge WebDriver
driver = webdriver.Edge(options=edge_options)

# Loop through pages 1 to 30 
for i in range(1, 30):
    url = f'https://my.lazada.co.th/pdp/review/getReviewList?itemId={itemid}&pageSize=50&filter=0&sort=0&pageNo={i}'
    try:
        driver.get(url)
        connection_ok = True
    except WebDriverException as e:
        print(f"❌ Cannot load page {i} — Error: {e}")
        connection_ok = False

    # ----- Use if-else logic based on connection status -----
    if connection_ok:
        hidden_div = scrape_review(driver)

        if hidden_div is None:
            input("⚠️ CAPTCHA triggered. Please solve manually and press Enter...")
            hidden_div = scrape_review(driver)

        if hidden_div:
            json_string = hidden_div.text

            try:
                data = json.loads(json_string)

                if data and "model" in data and data["model"] and "items" in data["model"]:
                    reviews = data["model"]["items"]
                    number_of_reviews = len(reviews)
                    print(f"Page {i}: Number of reviews: {number_of_reviews}")

                    if number_of_reviews == 0:
                        print("✅ No more reviews. Exiting...")
                        break

                    reviews_list = [
                        {
                            "username": r.get("buyerName", ""),
                            "options": r.get("skuInfo", ""),
                            "comment": r.get("reviewContent", ""),
                            "rating": r.get("rating", ""),
                            "date": r.get("reviewTime", ""),
                            "product": r.get("itemTitle", "")
                        }
                        for r in reviews
                    ]

                    df_page = pd.DataFrame(reviews_list)
                    df = pd.concat([df, df_page], ignore_index=True)

                else:
                    print(f"⚠️ Invalid JSON structure on page {i}")
                    break

            except json.JSONDecodeError as e:
                print(f"🚫 Failed to decode JSON on page {i}: {e}")
                break
            except Exception as e:
                print(f"🚨 Unexpected error on page {i}: {e}")
                break

        time.sleep(random.uniform(5, 10))  # Random delay
    else:
        print("🔌 Connection to WebDriver failed. Exiting scraping loop.")
        break

# Try to close the driver safely
try:
    driver.quit()
except:
    pass

# Final cleanup
df = df.drop_duplicates()

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_138_WINDOWS)); using driver found in the cache


⚠️ CAPTCHA triggered. Please solve manually and press Enter... 


Page 1: Number of reviews: 50
Page 2: Number of reviews: 50
Page 3: Number of reviews: 50
Page 4: Number of reviews: 16
Page 5: Number of reviews: 0
✅ No more reviews. Exiting...


In [22]:
df['product'] = 'SRICHAND Skin Moisture Burst Essence'
df['source'] = 'Lazada'

In [23]:
from datetime import datetime

# Lists for mapping Thai month abbreviations to English
THAI_ABBR_MONTHS = {
    "ม.ค.": "Jan", "ก.พ.": "Feb", "มี.ค.": "Mar", "เม.ย.": "Apr", "พ.ค.": "May", "มิ.ย.": "Jun",
    "ก.ค.": "Jul", "ส.ค.": "Aug", "ก.ย.": "Sep", "ต.ค.": "Oct", "พ.ย.": "Nov", "ธ.ค.": "Dec"
}

# Function to convert Thai date string to Python datetime object
def thai_date_to_datetime(thai_date_str):
    for thai_abbr, eng_abbr in THAI_ABBR_MONTHS.items():
        if thai_abbr in thai_date_str:
            # Replace Thai month abbreviation with English month abbreviation
            translated_date_str = thai_date_str.replace(thai_abbr, eng_abbr)
            # Convert the string to a datetime object
            return datetime.strptime(translated_date_str, "%d %b %Y")
    # If no conversion was made, return None or raise an exception
    return None


In [24]:
# Transform date column 
# Handle date format 

from datetime import datetime, timedelta

reference_date = datetime.now()

# Dictionary to map durations to corresponding timedelta objects
duration_mapping = {
    'ชั่วโมงที่แล้ว': timedelta(hours=-1),
    'นาทีที่แล้ว': timedelta(minutes=-1),
    'วันที่แล้ว': timedelta(days=-1),
    'สัปดาห์ที่แล้ว': timedelta(weeks=-1),
}
# Function to convert time intervals (e.g., "2 hours ago") to actual dates
def convert_time_interval(interval):
    # Handle the "Today" case
    if interval == 'วันนี้':  # 'วันนี้' means 'Today' in Thai
        return reference_date.strftime("%d/%m/%Y")
    
    # Handle relative time intervals (e.g., "2 hours ago")
    parts = interval.split()
    
    if len(parts) == 2:  # Case like "2 วันก่อน" (2 days ago)
        quantity = int(parts[0])
        time_unit = ' '.join(parts[1:])
        
        # Find the matching time duration (hours, days, weeks, etc.)
        if time_unit in duration_mapping:
            return (reference_date + duration_mapping[time_unit] * quantity).strftime("%d/%m/%Y")
    
    # Handle full Thai date like "12 ก.พ. 2023"
    return thai_date_to_datetime(interval).strftime("%d/%m/%Y")
    
# Assuming df['date'] contains the time intervals
df['date'] = df['date'].apply(convert_time_interval)

In [25]:
df

,username,options,comment,rating,date,product,source
0,J***n,None,Price: จัดส่งไวมากกกกกกกกกกกกกก สินค้ามีคุณภา...,5,26/03/2025,SRICHAND Skin Moisture Burst Essence,Lazada
1,D***a,None,ร้านค้าจัดส่งสินค้าให้เร็วมาก แพ็คสินค้าดีมากห...,5,05/04/2025,SRICHAND Skin Moisture Burst Essence,Lazada
2,ธ***.,None,"ให้ความชุ่มชื้นแก่ผิวอย่างล้ำลึก, ความชุ่มชื้น...",5,06/04/2025,SRICHAND Skin Moisture Burst Essence,Lazada
3,Phatthama S.,None,"ให้ผิวเปล่งประกายสุขภาพดี, ให้ความชุ่มชื้นแก่ผ...",5,07/04/2025,SRICHAND Skin Moisture Burst Essence,Lazada
4,Khwan,None,"ให้ผิวเปล่งประกายสุขภาพดี, แบรนด์ Srichand ไม่...",5,02/04/2025,SRICHAND Skin Moisture Burst Essence,Lazada
...,...,...,...,...,...,...,...
161,p***k,None,"ช่วยรักษาความชุ่มชื้นให้ผิวตลอดวัน, เติมความชุ...",5,16/04/2024,SRICHAND Skin Moisture Burst Essence,Lazada
162,AEKKAPHONG T.,None,ส่งเร็ว,5,30/08/2024,SRICHAND Skin Moisture Burst Essence,Lazada
163,ภรภิรมย์ ล.,None,ไม่แพ้ค่ะแต่ได้น้อยจังซองนึงมีนิดเดียว😊,5,24/04/2024,SRICHAND Skin Moisture Burst Essence,Lazada
164,F***.,None,None,5,26/11/2024,SRICHAND Skin Moisture Burst Essence,Lazada


In [1]:
# df = df.astype(str).applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
# df.to_excel("SRICHAND Skin Moisture Burst Essence-3.xlsx", index=False)

In [24]:
### Lazada

In [56]:
import pandas as pd

df1 = pd.read_excel('SILKYGIRL/Silkygirl Instance Pore Fix Primer -lazada.xlsx')
df2 = pd.read_excel('SILKYGIRL/SILKYGIRL Instant Pore Fix Primer (15ml).xlsx')


In [57]:
df1.columns

Index(['date', 'middle', 'content', 'skuInfo'], dtype='object')

In [58]:
df1.rename(columns={'middle':'username','content':'comment','skuInfo':'options'},inplace=True)

In [59]:
df1['source'] = 'Lazada'
df2['source'] = 'Shopee'

In [60]:
df2.columns

Index(['username', 'rating', 'comment', 'date', 'product_selected', 'name',
       'options', 'source'],
      dtype='object')

In [61]:
df_all = pd.concat([df2,df1],axis=0)
df_all

,username,rating,comment,date,product_selected,name,options,source
0,a*****z,5.0,repeated use using this brand. it worked well ...,2025-02-10 22:43:21,"[{'itemid': 3412756752, 'shopid': 74377558, 'n...",SILKYGIRL Instant Pore Fix Primer (15ml),['01 Natural'],Shopee
1,iqah_iqah,5.0,Usage Experience:make up not crack\nUser Tips:...,2025-01-14 10:56:03,"[{'itemid': 3412756752, 'shopid': 74377558, 'n...",SILKYGIRL Instant Pore Fix Primer (15ml),['01 Natural'],Shopee
2,s*****i,5.0,Texture:8/10\nCoverage:8/10\n\nPenghantaran pa...,2025-01-09 23:23:24,"[{'itemid': 3412756752, 'shopid': 74377558, 'n...",SILKYGIRL Instant Pore Fix Primer (15ml),['01 Natural'],Shopee
3,88ec_gapc2,5.0,"Dah sampai, penghantaran laju juga\nLgi murah ...",2025-02-10 14:15:52,"[{'itemid': 3412756752, 'shopid': 74377558, 'n...",SILKYGIRL Instant Pore Fix Primer (15ml),['01 Natural'],Shopee
4,e*****6,5.0,"barang dah sampai dengan selamat, banyak diska...",2025-02-02 09:05:16,"[{'itemid': 3412756752, 'shopid': 74377558, 'n...",SILKYGIRL Instant Pore Fix Primer (15ml),['01 Natural'],Shopee
...,...,...,...,...,...,...,...,...
205,Hanes,NaN,good condition,2020-11-18 00:00:00,NaN,NaN,Color:01 Natural,Lazada
206,Siti N.,NaN,My 1st time bought this. Hopefully it turn out...,2020-11-18 00:00:00,NaN,NaN,Color:01 Natural,Lazada
207,M***.,NaN,received in good condition but haven't try it yet,2020-04-12 00:00:00,NaN,NaN,"Color:01 Natural, Volume (ml):15",Lazada
208,C***.,NaN,Not really can cover !,2021-11-15 00:00:00,NaN,NaN,Color:01 Natural,Lazada


In [62]:
df_all.to_excel('Silkygirl Instance Pore Fix Primer - all.xlsx',index=False)